In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
#from google.colab import files
#uploaded = files.upload()

In [2]:
#import io
#df = pd.read_csv(io.BytesIO(uploaded['nba_2017_nba_players_with_salary.csv']))
data = pd.read_csv('nba_2017_nba_players_with_salary.csv')
data.describe()

,Unnamed: 0,Rk,AGE,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,POINTS,GP,MPG,ORPM,DRPM,RPM,WINS_RPM,PIE,PACE,W,SALARY_MILLIONS
count,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,320.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.00000,337.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000
mean,170.500000,217.269006,26.444444,21.572515,3.483626,7.725439,0.446096,0.865789,2.440058,0.307016,2.620175,5.282456,0.486749,0.499336,1.621345,2.09152,0.748484,0.899123,2.987719,3.885380,2.064035,0.678947,0.417836,1.227485,1.789766,9.446491,58.198830,21.572807,-0.676023,-0.005789,-0.681813,2.861725,9.186842,98.341053,28.950292,7.294006
std,98.871128,136.403138,4.295686,8.804018,2.200872,4.646933,0.078992,0.780010,2.021716,0.134691,1.828714,3.531233,0.084118,0.076316,1.532669,1.83971,0.115233,0.756961,1.799029,2.408031,1.892033,0.410237,0.420031,0.826223,0.700428,6.262225,22.282015,8.804121,2.063237,1.614293,2.522014,3.880914,3.585475,2.870091,14.603876,6.516326
min,0.000000,1.000000,19.000000,2.200000,0.000000,0.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.00000,0.273000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,2.000000,2.200000,-4.430000,-3.920000,-6.600000,-2.320000,-1.600000,87.460000,0.000000,0.030000
25%,85.250000,100.250000,23.000000,15.025000,1.800000,4.225000,0.402250,0.200000,0.800000,0.280250,1.200000,2.600000,0.446250,0.471000,0.600000,0.90000,0.686000,0.400000,1.700000,2.125000,0.800000,0.400000,0.100000,0.700000,1.300000,5.000000,43.500000,15.025000,-2.147500,-1.222500,-2.422500,0.102500,7.100000,96.850000,19.000000,2.185000
50%,170.500000,205.500000,26.000000,21.650000,3.000000,6.700000,0.442000,0.700000,2.200000,0.340500,2.200000,4.300000,0.487000,0.502000,1.200000,1.60000,0.768000,0.700000,2.650000,3.300000,1.500000,0.600000,0.300000,1.000000,1.800000,7.950000,66.000000,21.650000,-0.990000,-0.130000,-1.170000,1.410000,8.700000,98.205000,29.000000,4.920000
75%,255.750000,327.750000,29.000000,29.075000,4.700000,10.400000,0.481000,1.400000,3.600000,0.373500,3.700000,7.600000,0.528750,0.533000,2.000000,2.70000,0.833000,1.200000,3.800000,5.100000,2.600000,0.900000,0.500000,1.600000,2.300000,12.775000,76.000000,29.075000,0.257500,1.067500,0.865000,4.487500,10.900000,100.060000,39.000000,11.110000
max,341.000000,482.000000,40.000000,37.800000,10.300000,24.000000,0.750000,4.100000,10.000000,1.000000,9.700000,18.600000,1.000000,0.875000,9.200000,10.90000,1.000000,4.000000,9.500000,12.800000,11.200000,2.000000,2.600000,5.700000,3.900000,31.600000,82.000000,37.800000,7.270000,6.020000,8.420000,20.430000,23.000000,109.870000,66.000000,30.960000


In [0]:
# 刪除重複的球員(數據也都一樣)
data = data.drop_duplicates(['PLAYER'],keep='last')
data.reset_index(drop=True, inplace=True)

In [4]:
# 以3個四分位數2.185, 4.92, 11.11切
salary = data['SALARY_MILLIONS']
# print(salary)

salary_class = []
for i in salary:
    if i < 2.185:
        salary_class.append(0)
    elif 2.185<=i and 4.92>=i:
        salary_class.append(1)
    else:
        salary_class.append(2)

# 先不加入 等標準化後再加入        
data['SALARY_CLASS'] = salary_class
data.describe()

,Unnamed: 0,Rk,AGE,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,POINTS,GP,MPG,ORPM,DRPM,RPM,WINS_RPM,PIE,PACE,W,SALARY_MILLIONS,SALARY_CLASS
count,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,314.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,330.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000
mean,168.134328,213.829851,26.507463,21.818209,3.526866,7.822687,0.445122,0.878507,2.474030,0.305803,2.650746,5.345970,0.485857,0.498546,1.644776,2.118507,0.752118,0.906567,3.018209,3.923881,2.090149,0.688358,0.423582,1.243582,1.808955,9.569851,58.644776,21.818507,-0.645403,0.007134,-0.638269,2.919075,9.183881,98.384448,29.170149,7.415313,1.274627
std,98.306056,135.300943,4.275586,8.710201,2.200393,4.639471,0.077781,0.782567,2.027287,0.129292,1.833175,3.536081,0.084152,0.074061,1.539702,1.848875,0.112278,0.760959,1.799167,2.410102,1.900628,0.408754,0.421871,0.826224,0.691768,6.263134,22.076536,8.710298,2.072302,1.627403,2.528990,3.900669,3.564283,2.826068,14.548628,6.529251,0.820279
min,0.000000,1.000000,19.000000,2.200000,0.000000,0.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.273000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,2.000000,2.200000,-4.430000,-3.920000,-6.600000,-2.320000,-1.600000,87.460000,0.000000,0.030000,0.000000
25%,83.500000,98.500000,23.000000,15.500000,1.900000,4.400000,0.402000,0.200000,0.800000,0.281500,1.200000,2.600000,0.446000,0.471000,0.600000,0.900000,0.692000,0.400000,1.750000,2.200000,0.800000,0.400000,0.100000,0.700000,1.400000,5.100000,46.000000,15.500000,-2.120000,-1.215000,-2.345000,0.135000,7.100000,96.860000,19.000000,2.290000,1.000000
50%,167.000000,202.000000,26.000000,21.900000,3.000000,6.800000,0.442000,0.700000,2.300000,0.341000,2.200000,4.400000,0.486000,0.502000,1.200000,1.600000,0.769000,0.700000,2.700000,3.400000,1.500000,0.600000,0.300000,1.000000,1.800000,8.100000,67.000000,21.900000,-0.920000,-0.070000,-1.150000,1.480000,8.700000,98.270000,29.000000,5.000000,2.000000
75%,252.500000,323.500000,29.000000,29.200000,4.750000,10.500000,0.480500,1.400000,3.700000,0.374500,3.800000,7.650000,0.528000,0.533000,2.000000,2.700000,0.835250,1.200000,3.800000,5.100000,2.600000,0.900000,0.500000,1.600000,2.300000,12.800000,76.000000,29.200000,0.290000,1.075000,0.895000,4.630000,10.900000,100.060000,39.000000,11.235000,2.000000
max,341.000000,482.000000,40.000000,37.800000,10.300000,24.000000,0.750000,4.100000,10.000000,1.000000,9.700000,18.600000,1.000000,0.875000,9.200000,10.900000,1.000000,4.000000,9.500000,12.800000,11.200000,2.000000,2.600000,5.700000,3.900000,31.600000,82.000000,37.800000,7.270000,6.020000,8.420000,20.430000,23.000000,109.870000,66.000000,30.960000,2.000000


In [5]:
# 刪除不需要之features
data = data.drop(['MP','Unnamed: 0','Rk'], axis=1)
# 3P%、FG%原有空白值 將空白值以0取代
data = data.fillna(0)
# 重新排序col順序(前三個部用加入分析)
data = data[['PLAYER', 'TEAM', 'POSITION', 'AGE', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'POINTS', 'GP', 'MPG', 'ORPM',
       'DRPM', 'RPM', 'WINS_RPM', 'PIE', 'PACE', 'W', 'SALARY_MILLIONS', 'SALARY_CLASS']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 37 columns):
PLAYER             335 non-null object
TEAM               335 non-null object
POSITION           335 non-null object
AGE                335 non-null int64
FG                 335 non-null float64
FGA                335 non-null float64
FG%                335 non-null float64
3P                 335 non-null float64
3PA                335 non-null float64
3P%                335 non-null float64
2P                 335 non-null float64
2PA                335 non-null float64
2P%                335 non-null float64
eFG%               335 non-null float64
FT                 335 non-null float64
FTA                335 non-null float64
FT%                335 non-null float64
ORB                335 non-null float64
DRB                335 non-null float64
TRB                335 non-null float64
AST                335 non-null float64
STL                335 non-null float64
BLK                3

In [6]:
data.columns

Index(['PLAYER', 'TEAM', 'POSITION', 'AGE', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'POINTS', 'GP', 'MPG', 'ORPM',
       'DRPM', 'RPM', 'WINS_RPM', 'PIE', 'PACE', 'W', 'SALARY_MILLIONS',
       'SALARY_CLASS'],
      dtype='object')

In [0]:
data1 = data[['PLAYER', 'TEAM', 'POSITION']]
data2 = data[['AGE', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'POINTS', 'GP', 'MPG', 'ORPM',
       'DRPM', 'RPM', 'WINS_RPM', 'PIE', 'PACE', 'W', 'SALARY_MILLIONS']]

In [0]:
scale = preprocessing.MinMaxScaler()
data2 = pd.DataFrame(scale.fit_transform(data2),
                                columns=data2.keys())
data = pd.concat([data1,data2], axis = 1)

In [9]:
data['SALARY_CLASS'] = salary_class
data

,PLAYER,TEAM,POSITION,AGE,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,POINTS,GP,MPG,ORPM,DRPM,RPM,WINS_RPM,PIE,PACE,W,SALARY_MILLIONS,SALARY_CLASS
0,Russell Westbrook,OKC,PG,0.428571,0.990291,1.000000,0.566667,0.609756,0.72,0.343,0.793814,0.902174,0.459,0.544000,0.956522,0.954128,0.845,0.425,0.947368,0.835938,0.928571,0.80,0.153846,0.947368,0.589744,1.000000,0.9875,0.910112,0.954701,0.347082,0.856858,0.864176,1.000000,0.662651,0.696970,0.855803,2
1,James Harden,HOU,PG,0.380952,0.805825,0.780172,0.586667,0.780488,0.93,0.347,0.525773,0.510870,0.530,0.600000,1.000000,1.000000,0.847,0.300,0.736842,0.632812,1.000000,0.75,0.192308,1.000000,0.692308,0.920382,0.9875,0.960674,0.923932,0.236419,0.759654,0.785055,0.837398,0.692548,0.818182,0.855803,2
2,Isaiah Thomas,BOS,PG,0.380952,0.873786,0.801724,0.617333,0.780488,0.85,0.379,0.597938,0.581522,0.528,0.624000,0.847826,0.779817,0.909,0.150,0.221053,0.210938,0.526786,0.45,0.076923,0.491228,0.564103,0.914013,0.9250,0.887640,0.867521,0.003018,0.561252,0.461978,0.719512,0.552432,0.772727,0.212092,2
3,Anthony Davis,NO,C,0.190476,1.000000,0.840517,0.673333,0.121951,0.18,0.299,1.000000,1.000000,0.524,0.592000,0.750000,0.788991,0.802,0.575,1.000000,0.921875,0.187500,0.65,0.846154,0.421053,0.564103,0.885350,0.9125,0.952247,0.417094,0.786720,0.729028,0.665055,0.845528,0.568050,0.469697,0.714193,2
4,DeMarcus Cousins,NO/SAC,C,0.333333,0.873786,0.823276,0.602667,0.439024,0.50,0.361,0.742268,0.793478,0.483,0.569143,0.782609,0.853211,0.772,0.525,0.936842,0.859375,0.410714,0.70,0.500000,0.649123,1.000000,0.853503,0.8750,0.898876,0.682906,0.458753,0.719041,0.596923,0.788618,0.430611,0.454545,0.547365,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,Brice Johnson,LAC,PF,0.142857,0.067961,0.064655,0.381333,0.000000,0.00,0.000,0.072165,0.114130,0.286,0.326857,0.000000,0.000000,0.000,0.075,0.073684,0.078125,0.026786,0.35,0.115385,0.052632,0.000000,0.035032,0.0125,0.022472,0.180342,0.503018,0.356858,0.102418,0.504065,0.368585,0.015152,0.042030,0
331,Mike Tobey,CHA,C,0.142857,0.048544,0.051724,0.333333,0.000000,0.00,0.000,0.051546,0.097826,0.250,0.285714,0.000000,0.000000,0.000,0.250,0.052632,0.117188,0.044643,0.00,0.000000,0.087719,0.256410,0.025478,0.0000,0.289326,0.215385,0.493964,0.378162,0.103297,0.000000,0.176261,0.000000,0.001293,0
332,Alonzo Gee,DEN,SF,0.476190,0.019417,0.012931,0.285333,0.000000,0.02,0.000,0.020619,0.032609,0.273,0.244571,0.043478,0.064220,0.556,0.075,0.084211,0.093750,0.044643,0.20,0.038462,0.052632,0.282051,0.019108,0.1375,0.129213,0.198291,0.437626,0.327563,0.105495,0.097561,0.752789,0.060606,0.000970,0
333,John Lucas III,MIN,PG,0.714286,0.019417,0.000000,0.333333,0.000000,0.02,0.000,0.020619,0.021739,0.333,0.285714,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.017857,0.20,0.000000,0.000000,0.000000,0.006369,0.0375,0.000000,0.267521,0.332998,0.312250,0.102418,0.361789,0.312807,0.045455,0.030715,0
